# Customer Churn Prediction Model
# Real-world dataset analysis w
# Author: Peter Chika Ozo-ogueji (Data Scientist)
# Tech Stack: Python, Scikit-learn, XGBoost, TensorFlow, Pandas

In [1]:
# Customer Churn Prediction Model
# Real-world dataset analysis
# Author: Peter Chika Ozo-ogueji (Data Scientist)
# Tech Stack: Python, Scikit-learn, XGBoost, TensorFlow, Pandas

"""
🎯 PROJECT OVERVIEW - CORRECTED VERSION
=========================================
This project demonstrates advanced machine learning techniques for customer churn prediction.
FIXES APPLIED:
- Corrected neural network architecture
- Realistic business assumptions
- Proper cross-validation methodology
- Accurate performance metrics
- Professional code structure
"""

# ============================================================================
# 1. ENVIRONMENT SETUP & DATA IMPORT
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import precision_recall_curve, f1_score, accuracy_score, precision_score, recall_score
from imblearn.over_sampling import SMOTE

import xgboost as xgb

# Deep Learning with CORRECTED architecture
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

plt.style.use('default')
sns.set_palette("husl")

print("🚀 Environment setup complete! Fixed version ready.")

# ============================================================================
# 2. DATA LOADING & INITIAL EXPLORATION
# ============================================================================

print("📊 Loading real-world customer churn dataset...")

# Load real IBM Telco dataset (NOT synthetic)
url = "https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv"
df = pd.read_csv(url)

print(f"✅ Dataset loaded successfully!")
print(f"📋 Dataset shape: {df.shape}")
print(f"🎯 Target variable: Churn (Yes/No)")

# Initial data exploration
print("\n" + "="*60)
print("📈 INITIAL DATA EXPLORATION")
print("="*60)

print(f"\nTarget distribution:")
print(df['Churn'].value_counts())
print(f"Churn rate: {df['Churn'].value_counts(normalize=True)['Yes']:.2%}")

print(f"\nMissing values:")
missing_values = df.isnull().sum().sort_values(ascending=False)
print(missing_values[missing_values > 0])

# ============================================================================
# 3. DATA PREPROCESSING & FEATURE ENGINEERING
# ============================================================================

print("\n" + "="*60)
print("🔧 DATA PREPROCESSING & FEATURE ENGINEERING")
print("="*60)

# Create a copy for processing
df_processed = df.copy()

print("🧹 Cleaning data...")

# FIXED: Proper handling of TotalCharges
df_processed['TotalCharges'] = pd.to_numeric(df_processed['TotalCharges'], errors='coerce')
# Fill missing values with 0 for new customers (more realistic assumption)
df_processed['TotalCharges'].fillna(0, inplace=True)

# Convert target variable to binary
df_processed['Churn'] = df_processed['Churn'].map({'Yes': 1, 'No': 0})

# CORRECTED Feature Engineering with realistic assumptions
print("🎨 Creating corrected features...")

# 1. Tenure categories (more business-realistic)
df_processed['TenureGroup'] = pd.cut(df_processed['tenure'],
                                   bins=[0, 6, 18, 36, 100],
                                   labels=['New', 'Short', 'Medium', 'Long'])

# 2. Service count (simplified and accurate)
service_cols = ['PhoneService', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
               'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

df_processed['ServiceCount'] = 0
for col in service_cols:
    if col == 'PhoneService':
        df_processed['ServiceCount'] += (df_processed[col] == 'Yes').astype(int)
    elif col == 'InternetService':
        df_processed['ServiceCount'] += (df_processed[col] != 'No').astype(int)
    else:
        df_processed['ServiceCount'] += (df_processed[col] == 'Yes').astype(int)

# 3. Realistic financial metrics
df_processed['AvgMonthlyCharges'] = np.where(df_processed['tenure'] > 0,
                                           df_processed['TotalCharges'] / df_processed['tenure'],
                                           df_processed['MonthlyCharges'])

# 4. Payment and contract features
df_processed['AutoPay'] = (df_processed['PaymentMethod'].isin([
    'Bank transfer (automatic)', 'Credit card (automatic)'])).astype(int)

df_processed['MonthToMonth'] = (df_processed['Contract'] == 'Month-to-month').astype(int)

# 5. Security services indicator
df_processed['HasSecurity'] = ((df_processed['OnlineSecurity'] == 'Yes') |
                              (df_processed['OnlineBackup'] == 'Yes') |
                              (df_processed['DeviceProtection'] == 'Yes')).astype(int)

print("✅ features created successfully!")

# ============================================================================
# 4. FEATURE PREPROCESSING FOR MACHINE LEARNING
# ============================================================================

print("\n" + "="*60)
print("🤖 FEATURE PREPROCESSING")
print("="*60)

# Select features for modeling (reduced set for better performance)
categorical_features = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
                       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
                       'PaperlessBilling', 'PaymentMethod', 'TenureGroup']

numerical_features = ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges',
                     'ServiceCount', 'AvgMonthlyCharges', 'AutoPay', 'MonthToMonth', 'HasSecurity']

# Encode categorical variables
df_ml = df_processed.copy()
label_encoders = {}

for feature in categorical_features:
    if feature in df_ml.columns:
        le = LabelEncoder()
        df_ml[f'{feature}_encoded'] = le.fit_transform(df_ml[feature].astype(str))
        label_encoders[feature] = le

# Prepare feature matrix
encoded_features = [f'{f}_encoded' for f in categorical_features if f in df_ml.columns]
all_features = numerical_features + encoded_features
feature_columns = [col for col in all_features if col in df_ml.columns]

X = df_ml[feature_columns]
y = df_ml['Churn']

print(f"📋 Selected {len(feature_columns)} features for modeling")
print(f"✅ Feature matrix shape: {X.shape}")
print(f"✅ Target distribution: {y.value_counts().to_dict()}")

# ============================================================================
# 5. MODEL DEVELOPMENT & TRAINING
# ============================================================================

print("\n" + "="*60)
print("🎯 MODEL DEVELOPMENT & TRAINING")
print("="*60)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

print(f"🔹 Training set: {X_train.shape[0]} samples")
print(f"🔹 Validation set: {X_val.shape[0]} samples")
print(f"🔹 Test set: {X_test.shape[0]} samples")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Handle class imbalance with SMOTE (moderate approach)
smote = SMOTE(random_state=42, sampling_strategy=0.8)  # Less aggressive balancing
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"✅ After moderate SMOTE - Training set: {X_train_balanced.shape[0]} samples")

models_results = {}

# ============================================================================
# 5.1 RANDOM FOREST CLASSIFIER
# ============================================================================

print("\n🌲 Training Random Forest Classifier...")

# Simplified parameter grid for realistic training time
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 15, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_grid = GridSearchCV(rf, rf_params, cv=3, scoring='roc_auc', n_jobs=-1)
rf_grid.fit(X_train_balanced, y_train_balanced)

best_rf = rf_grid.best_estimator_
rf_pred = best_rf.predict(X_test)
rf_pred_proba = best_rf.predict_proba(X_test)[:, 1]

models_results['Random Forest'] = {
    'model': best_rf,
    'predictions': rf_pred,
    'probabilities': rf_pred_proba,
    'accuracy': accuracy_score(y_test, rf_pred),
    'roc_auc': roc_auc_score(y_test, rf_pred_proba),
    'f1': f1_score(y_test, rf_pred),
    'precision': precision_score(y_test, rf_pred),
    'recall': recall_score(y_test, rf_pred)
}

print(f"✅ Random Forest - ROC AUC: {models_results['Random Forest']['roc_auc']:.4f}")

# ============================================================================
# 5.2 XGBOOST CLASSIFIER
# ============================================================================

print("\n🚀 Training XGBoost Classifier...")

# Simplified XGBoost parameters
xgb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.8, 1.0]
}

xgb_model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
xgb_grid = GridSearchCV(xgb_model, xgb_params, cv=3, scoring='roc_auc', n_jobs=-1)
xgb_grid.fit(X_train_balanced, y_train_balanced)

best_xgb = xgb_grid.best_estimator_
xgb_pred = best_xgb.predict(X_test)
xgb_pred_proba = best_xgb.predict_proba(X_test)[:, 1]

models_results['XGBoost'] = {
    'model': best_xgb,
    'predictions': xgb_pred,
    'probabilities': xgb_pred_proba,
    'accuracy': accuracy_score(y_test, xgb_pred),
    'roc_auc': roc_auc_score(y_test, xgb_pred_proba),
    'f1': f1_score(y_test, xgb_pred),
    'precision': precision_score(y_test, xgb_pred),
    'recall': recall_score(y_test, xgb_pred)
}

print(f"✅ XGBoost - ROC AUC: {models_results['XGBoost']['roc_auc']:.4f}")

# ============================================================================
# 5.3 NEURAL NETWORK
# ============================================================================

print("\n🧠 Training CORRECTED Neural Network...")

# FIXED: Much simpler architecture to prevent overfitting
def create_simple_neural_network(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_dim=input_dim),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    # FIXED: Lower learning rate and proper compilation
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'AUC'])

    return model

# Create and train corrected neural network
nn_model = create_simple_neural_network(X_train_scaled.shape[1])

# FIXED: Proper early stopping
early_stopping = EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True, mode='max')

# FIXED: Scale the balanced data properly for neural network
X_train_balanced_scaled = scaler.fit_transform(X_train_balanced)

# Train with corrected parameters
history = nn_model.fit(
    X_train_balanced_scaled, y_train_balanced,
    validation_data=(X_val_scaled, y_val),
    epochs=50,  # FIXED: Reduced epochs
    batch_size=64,  # FIXED: Larger batch size
    callbacks=[early_stopping],
    verbose=0  # Reduced verbosity
)

# Make predictions
nn_pred_proba = nn_model.predict(X_test_scaled, verbose=0).flatten()
nn_pred = (nn_pred_proba > 0.5).astype(int)

models_results['Neural Network'] = {
    'model': nn_model,
    'predictions': nn_pred,
    'probabilities': nn_pred_proba,
    'accuracy': accuracy_score(y_test, nn_pred),
    'roc_auc': roc_auc_score(y_test, nn_pred_proba),
    'f1': f1_score(y_test, nn_pred),
    'precision': precision_score(y_test, nn_pred),
    'recall': recall_score(y_test, nn_pred)
}

print(f"✅ Neural Network - ROC AUC: {models_results['Neural Network']['roc_auc']:.4f}")

# ============================================================================
# 6. MODEL EVALUATION & COMPARISON
# ============================================================================

print("\n" + "="*60)
print("📊 MODEL EVALUATION & COMPARISON")
print("="*60)

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': list(models_results.keys()),
    'Accuracy': [models_results[model]['accuracy'] for model in models_results.keys()],
    'ROC AUC': [models_results[model]['roc_auc'] for model in models_results.keys()],
    'F1 Score': [models_results[model]['f1'] for model in models_results.keys()],
    'Precision': [models_results[model]['precision'] for model in models_results.keys()],
    'Recall': [models_results[model]['recall'] for model in models_results.keys()]
})

comparison_df = comparison_df.sort_values('ROC AUC', ascending=False)
print("\n🏆 MODEL PERFORMANCE COMPARISON:")
print("="*50)
print(comparison_df.round(4))

# Select best model
best_model_name = comparison_df.iloc[0]['Model']
best_model = models_results[best_model_name]['model']

print(f"\n🏆 BEST PERFORMING MODEL: {best_model_name}")
print(f"✅ Accuracy: {models_results[best_model_name]['accuracy']:.4f}")
print(f"✅ ROC AUC: {models_results[best_model_name]['roc_auc']:.4f}")
print(f"✅ F1 Score: {models_results[best_model_name]['f1']:.4f}")

# ============================================================================
# 7. BUSINESS IMPACT ANALYSIS
# ============================================================================

print("\n" + "="*60)
print("💰 BUSINESS IMPACT ANALYSIS")
print("="*60)

# Business assumptions
avg_customer_value = 1200  # Realistic annual customer value
retention_cost = 50       # Realistic retention campaign cost
acquisition_cost = 200    # Realistic acquisition cost
successful_retention_rate = 0.25  # Conservative 25% success rate

# Calculate business metrics with best model
best_predictions = models_results[best_model_name]['probabilities']
threshold = 0.5

# Confusion matrix components
tp = np.sum((best_predictions >= threshold) & (y_test == 1))
fp = np.sum((best_predictions >= threshold) & (y_test == 0))
tn = np.sum((best_predictions < threshold) & (y_test == 0))
fn = np.sum((best_predictions < threshold) & (y_test == 1))

print(f"📊 CONFUSION MATRIX ANALYSIS:")
print(f"True Positives (Correctly identified churners): {tp}")
print(f"False Positives (Incorrectly identified churners): {fp}")
print(f"True Negatives (Correctly identified retained): {tn}")
print(f"False Negatives (Missed churners): {fn}")

# Business value calculation
value_saved_from_retention = tp * successful_retention_rate * avg_customer_value
cost_of_campaigns = (tp + fp) * retention_cost
cost_of_missed_churners = fn * avg_customer_value * 0.3  # Only count partial loss

net_business_value = value_saved_from_retention - cost_of_campaigns

print(f"\n💰 BUSINESS VALUE ANALYSIS:")
print("="*40)
print(f"Average Customer Value: ${avg_customer_value:,.0f}")
print(f"Value saved from retention: ${value_saved_from_retention:,.0f}")
print(f"Cost of retention campaigns: ${cost_of_campaigns:,.0f}")
print(f"NET BUSINESS VALUE: ${net_business_value:,.0f}")

# ROI calculation
roi = ((net_business_value - cost_of_campaigns) / cost_of_campaigns) * 100 if cost_of_campaigns > 0 else 0
print(f"Return on Investment (ROI): {roi:.1f}%")

# ============================================================================
# 8.CROSS-VALIDATION & STABILITY ANALYSIS
# ============================================================================

print("\n" + "="*60)
print("🔍 MODEL VALIDATION")
print("="*60)

# FIXED: Proper cross-validation on original training data (not SMOTE'd)
cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='roc_auc')

print(f"📊 CROSS-VALIDATION ANALYSIS:")
print(f"Mean ROC AUC: {cv_scores.mean():.4f}")
print(f"Standard Deviation: {cv_scores.std():.4f}")
print(f"95% Confidence Interval: [{cv_scores.mean() - 1.96*cv_scores.std():.4f}, {cv_scores.mean() + 1.96*cv_scores.std():.4f}]")

# FIXED: Realistic stability assessment
if cv_scores.std() < 0.02:
    stability = "HIGH"
elif cv_scores.std() < 0.05:
    stability = "MODERATE"
else:
    stability = "LOW"

print(f"Model Stability: {stability}")

# ============================================================================
# 9. FEATURE IMPORTANCE ANALYSIS
# ============================================================================

if hasattr(best_model, 'feature_importances_'):
    print(f"\n🔍 TOP 10 FEATURE IMPORTANCES ({best_model_name}):")
    print("="*50)

    feature_importance = pd.DataFrame({
        'feature': feature_columns,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)

    for i, row in feature_importance.head(10).iterrows():
        print(f"{row['feature']}: {row['importance']:.4f}")

# ============================================================================
# 10. ACTIONABLE RECOMMENDATIONS
# ============================================================================

print("\n" + "="*60)
print("🎯 ACTIONABLE RECOMMENDATIONS")
print("="*60)

print("💼 STRATEGIC RECOMMENDATIONS:")
print("="*30)

print("\n1. 🎯 IMMEDIATE ACTIONS:")
print("   • Target month-to-month contract customers for retention")
print("   • Offer automatic payment incentives")
print("   • Implement early customer engagement programs")
print("   • Focus on customers with low service adoption")

print("\n2. 💰 REVENUE IMPACT:")
print(f"   • Deploy model to identify ~{int(tp+fp)} high-risk customers monthly")
print(f"   • Expected monthly value: ${net_business_value/12:,.0f}")
print(f"   • ROI of {roi:.1f}% on retention campaigns")

print("\n3. 🔧 IMPLEMENTATION:")
print("   • Integrate model into CRM system")
print("   • Set up monthly scoring and alerts")
print("   • A/B test retention strategies")
print("   • Monitor model performance quarterly")

print("\n4. 📊 MODEL MONITORING:")
print(f"   • Retrain if ROC AUC drops below {models_results[best_model_name]['roc_auc'] - 0.05:.3f}")
print("   • Monitor feature drift and data quality")
print("   • Track business KPIs and campaign effectiveness")

# ============================================================================
# 11. PROJECT SUMMARY
# ============================================================================

print("\n" + "="*60)
print("🏆 PROJECT SUMMARY")
print("="*60)

print("📋 TECHNICAL ACHIEVEMENTS:")
print("="*25)
print(f"✅ Achieved {models_results[best_model_name]['accuracy']:.1%} accuracy with {best_model_name}")
print(f"✅ ROC AUC score: {models_results[best_model_name]['roc_auc']:.4f}")
print(f"✅ Processed {len(df)} real customer records")
print(f"✅ Engineered {len(feature_columns)} meaningful features")
print(f"✅ Established realistic business value framework")

print(f"\n🎯 BUSINESS IMPACT:")
print("="*15)
print(f"• Monthly business value: ${net_business_value/12:,.0f}")
print(f"• Campaign ROI: {roi:.1f}%")
print(f"• Model precision: {models_results[best_model_name]['precision']:.1%}")
print(f"• Model recall: {models_results[best_model_name]['recall']:.1%}")

print(f"\n🔧 KEY FIXES APPLIED:")
print("="*20)
print("• Simplified neural network architecture (fixed overfitting)")
print("• Realistic business cost assumptions")
print("• Proper cross-validation methodology")
print("• Conservative performance estimates")
print("• Professional code structure and documentation")

print(f"\n" + "="*60)
print("🚀 PROJECT COMPLETE!")
print("="*60)
print("Ready for production deployment with realistic expectations.")

🚀 Environment setup complete! Fixed version ready.
📊 Loading real-world customer churn dataset...
✅ Dataset loaded successfully!
📋 Dataset shape: (7043, 21)
🎯 Target variable: Churn (Yes/No)

📈 INITIAL DATA EXPLORATION

Target distribution:
Churn
No     5174
Yes    1869
Name: count, dtype: int64
Churn rate: 26.54%

Missing values:
Series([], dtype: int64)

🔧 DATA PREPROCESSING & FEATURE ENGINEERING
🧹 Cleaning data...
🎨 Creating corrected features...
✅ features created successfully!

🤖 FEATURE PREPROCESSING
📋 Selected 25 features for modeling
✅ Feature matrix shape: (7043, 25)
✅ Target distribution: {0: 5174, 1: 1869}

🎯 MODEL DEVELOPMENT & TRAINING
🔹 Training set: 4225 samples
🔹 Validation set: 1409 samples
🔹 Test set: 1409 samples
✅ After moderate SMOTE - Training set: 5587 samples

🌲 Training Random Forest Classifier...
✅ Random Forest - ROC AUC: 0.8253

🚀 Training XGBoost Classifier...
✅ XGBoost - ROC AUC: 0.8315

🧠 Training CORRECTED Neural Network...
✅ Neural Network - ROC AUC: 0.